# Defining all libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -v 'nibabel==3.2.1'

import os
import random
import numpy as np
import nibabel as nib
from nilearn.image import resample_img

# Image directory and size parameters

In [ ]:
HOME_DIR = os.path.join("/content/drive/MyDrive/PROJECT/Segmentation/")

DATA_DIR = os.path.join(HOME_DIR,"Dataset")
NII_DIR = os.path.join(DATA_DIR, "CT-ORG")

OG_DIR = os.path.join(NII_DIR, "OG")
MODIFIED_VOXEL_DIR = os.path.join(NII_DIR, "MODIFIED_VOXEL")

In [ ]:
image_size = 256
IMG_DIM=(image_size,image_size,300)

# Resample NIfTI files

In [ ]:
def rescale_and_resample_nifti(input_file, output_file, target_shape=IMG_DIM):
    """
    Rescale and resample a NIfTI file to a target shape.

    :param nifti_file: Path to the NIfTI file to be rescaled and resampled.
    :type nifti_file: str
    :param target_shape: Target shape for the resampled image.
    :type target_shape: tuple, optional
    :return: Resampled NIfTI image.
    :rtype: nibabel.nifti1.Nifti1Image
    """
    
    # Load the NIfTI file
    img = nib.load(input_file)
    
    # Compute the rescaling factor for each dimension
    rescale_factor = np.array(img.shape) / np.array(target_shape)
    print('fatcor',rescale_factor)
    print('affine',np.diag(img.affine))
    
    # Compute the new affine matrix
    new_affine = img.affine.copy()
    #np.fill_diagonal(new_affine, np.diag(img.affine)[:3] * rescale_factor[:3])
    target_voxel=(2.0, 2.0, 5.0)
    np.fill_diagonal(new_affine, target_voxel + (1,))
    print('new_affine', new_affine)

    # Resample the image
    resampled_img = resample_img(img, target_affine=new_affine, target_shape=target_shape)
    
    nib.save(resampled_img, output_file)
    #print(f"resampled nifti file saved as {output_file}")


In [ ]:
for directory in os.listdir(OG_DIR):
    for sub_directory in os.listdir(os.path.join(OG_DIR, directory)):
        for file_name in os.listdir(os.path.join(OG_DIR, directory, sub_directory)):
            input_file = os.path.join(OG_DIR, directory, sub_directory, file_name)
            modified_voxel_path = os.path.join(MODIFIED_VOXEL_DIR, directory, sub_directory, file_name)
            if os.path.isfile(input_file):
                rescale_and_resample_nifti(input_file, modified_voxel_path)
            else:
                print('error', file_name)
                print('input_file >>', input_file)
                print('modified_voxel_path >>', modified_voxel_path)